In [1]:
# %reload_ext autoreload
# %autoreload all

from pathlib import Path

import pandas as pd

from src.objects import TaxationList, TopographicPlan
from src.processing.topographic_plan import create_topographic_plan
from src.processing.taxation_list import create_taxation_list

Константы для чтения объектов из dxf файла

In [2]:
NUMBERS_LAYERS = ["номера"]
LINES_LAYERS = ["полосы"]
CONTOURS_LAYERS = ["контуры"]
MIN_DISTANCE = 0.01

Чтение dxf файла топографического чертежа и создание объекта топографического плана

In [3]:
topographic_plan = create_topographic_plan(
    Path("data/example_topographic_plan.dxf"),
    NUMBERS_LAYERS, LINES_LAYERS, CONTOURS_LAYERS, MIN_DISTANCE)

Проверка вхождения номеров в фигуры

In [9]:
_numbers_shapes_df = pd.DataFrame(columns=['number_id', 'shape_id', 'number', 'type', 'geometry'])

for _, ids_row in topographic_plan._numbers_shapes_df.iterrows():
    number_id = ids_row['number_id']
    shape_id = ids_row['shape_id']
    number = topographic_plan._numbers_df.loc[number_id, 'number']
    shape_type = topographic_plan._shapes_df.loc[shape_id, 'type']
    shape_geom = topographic_plan._shapes_df.loc[shape_id, 'geometry']

    _numbers_shapes_df = pd.concat([
        _numbers_shapes_df,
        pd.DataFrame({
            'number_id': [number_id],
            'shape_id': [shape_id],
            'number': [number], 
            'type': [shape_type],
            'geometry': [shape_geom]
        })
    ], ignore_index=True)

_numbers_shapes_df

,number_id,shape_id,number,type,geometry
0,14,0,15,LineString,LINESTRING (3092.268998306507 1693.40414713145...
1,15,1,16,LineString,LINESTRING (3099.275403871401 1706.81442616184...
2,12,2,8,Polygon,"POLYGON ((3106.916523995033 1690.249262830753,..."
3,13,2,9,Polygon,"POLYGON ((3106.916523995033 1690.249262830753,..."
4,0,3,1,Polygon,"POLYGON ((3075.045526933311 1693.259229309259,..."
5,1,3,2,Polygon,"POLYGON ((3075.045526933311 1693.259229309259,..."
6,16,4,17,Polygon,"POLYGON ((3106.496601561342 1689.562929311654,..."


Проверка пересечения number_id между _trees_df и numbers_shapes_df

In [10]:
trees_number_ids = set(topographic_plan._trees_df['number_id'])
shapes_number_ids = set(_numbers_shapes_df['number_id'])

if len(trees_number_ids.intersection(shapes_number_ids)) > 0:
    raise ValueError("Обнаружено пересечение number_id между деревьями и фигурами")

Проверка, что все number_id из _numbers_df присутствуют либо в _trees_df либо в numbers_shapes_df

In [11]:
all_numbers_ids = set(topographic_plan._numbers_df.index)
used_number_ids = trees_number_ids.union(shapes_number_ids)

if len(all_numbers_ids - used_number_ids) > 0:
    raise ValueError("Обнаружены number_id, которые не используются ни в деревьях, ни в фигурах")

Константы для чтения ведомости таксации

In [7]:
IS_IMPORT_FIRST_ROW = False
NUMBER_COLUMN = 0
NAME_COLUMN = 1
QUANTITY_COLUMN = 2
HEIGHT_COLUMN = 3
DIAMETER_COLUMN = 4
QUALITY_COLUMN = 5

Чтение docx или xls ведомости таксации и создание объекта ведомости таксации

In [8]:
taxation_list_file = Path(r"C:\geowork\2024\Шарковщина\122.24 ЖД Шарковщина\122.24 Таксация.docx")

taxation_list = create_taxation_list(
    taxation_list_file,
    is_import_first_row=False,
    column_mapping={
        'number': NUMBER_COLUMN,
        'name': NAME_COLUMN,
        'quantity': QUANTITY_COLUMN,
        'height': HEIGHT_COLUMN,
        'diameter': DIAMETER_COLUMN,
        'quality': QUALITY_COLUMN
    }
)
taxation_list.table_data

KeyError: "['splitted_number'] not in index"